# CARGA DE NGROK 

In [0]:
#Download ngrok
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip

In [0]:
#ngrok is on :4040
get_ipython().system_raw('./ngrok http -region au 80 &')

In [4]:
#Retrieve public url
#!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

import os, sys, json
curl_json = os.popen("curl -s http://localhost:4040/api/tunnels").read()
ngrok_ip = json.loads(curl_json)['tunnels'][0]['public_url']
ngrok_ip

'https://1670de19.au.ngrok.io'

In [3]:
!lsof -i | grep "LISTEN"

node        10 root   11u  IPv6  16258      0t0  TCP *:8080 (LISTEN)
jupyter-n   26 root    3u  IPv4  20860      0t0  TCP 47f47afaa0a8:9000 (LISTEN)
python3   3133 root   27u  IPv4 109011      0t0  TCP localhost:42455 (LISTEN)
python3   3133 root   32u  IPv4 109015      0t0  TCP localhost:52315 (LISTEN)
python3   3133 root   35u  IPv4 109019      0t0  TCP localhost:49537 (LISTEN)
python3   3133 root   38u  IPv4 109023      0t0  TCP localhost:60181 (LISTEN)
python3   3133 root   48u  IPv4 109028      0t0  TCP localhost:46187 (LISTEN)
python3   3133 root   61u  IPv4 108375      0t0  TCP localhost:55657 (LISTEN)
ngrok     3155 root    7u  IPv4 109729      0t0  TCP localhost:4040 (LISTEN)


In [5]:
!pip install Flask==0.12.2
!pip install pymessenger==0.0.7.0

In [0]:
#Python libraries that we need to import for our bot
import random
from flask import Flask, request
from pymessenger.bot import Bot

app = Flask(__name__)
ACCESS_TOKEN = 'EAAGX9ZB00mvABAG6Xml8M58Rz4TCdZAVNIEf3P1IGeLhZCLH4ilnHkv2RxZAI7vK8ZBQrs98y3nTgKACnsCkRnfYOxg2QgqRtmZBP4Wf2Y27fFwSWjxnqCo9ePxFg15IiBWC8qMIXHhOrnbY2YzliXM9QLYccW3orXalbrHX8xTgZDZD'
VERIFY_TOKEN = 'secret'
bot = Bot(ACCESS_TOKEN)

#We will receive messages that Facebook sends our bot at this endpoint 
@app.route("/", methods=['GET', 'POST'])
def receive_message():
  if request.method == 'GET':
    """Before allowing people to message your bot, Facebook has implemented a verify token
    that confirms all requests that your bot receives came from Facebook.""" 
    token_sent = request.args.get("hub.verify_token")
    return verify_fb_token(token_sent)
  #if the request was not get, it must be POST and we can just proceed with sending a message back to user
  else:
    # get whatever message a user sent the bot
    output = request.get_json()
    for event in output['entry']:
      messaging = event['messaging']
      for message in messaging:
        if message.get('message'):
          #Facebook Messenger ID for user so we know where to send response back to
          recipient_id = message['sender']['id']
          if message['message'].get('text'):
            response_sent_text = get_message()
            send_message(recipient_id, response_sent_text)
          #if user sends us a GIF, photo,video, or any other non-text item
          if message['message'].get('attachments'):
            response_sent_nontext = get_message()
            send_message(recipient_id, response_sent_nontext)
  return "Message Processed"


def verify_fb_token(token_sent):
  #take token sent by facebook and verify it matches the verify token you sent
  #if they match, allow the request, else return an error
  if token_sent == VERIFY_TOKEN:
    return request.args.get("hub.challenge")
  return "Invalid verification token"

#uses PyMessenger to send response to user
def send_message(recipient_id, response):
  #sends user the text message provided via input response parameter
  bot.send_text_message(recipient_id, response)
  return "success"

# BASE DE DATOS

## ABRIR BASE DE DATOS

In [27]:
# Abrimos el documento
g = open('dataset_sample.txt','r',errors = 'ignore')
raw_g=g.read()
print(raw_g[0:4000])

Tienda Cuzco Vaso tropical 
paquete x 18 unidades, precio: 16 soles, stock 300 paquetes.

Tienda Humboldt Vaso tropical 
paquete x 12 unidades, precio: 12 soles, stock: 400 paquetes.

Tienda Puno Silla robusta alta 
paquete x 6 unidades, precio:  114 soles, stock: 500 paquetes.

Tienda Cuzco Banco robusto chato 
paquete x 12 unidades, precio: 104 soles, stock: 200 paquetes.

Tienda Humboldt Pinza cucharita 
paquete x 144 unidades, precio: 9 soles, stock: 1000 paquetes.

Tienda Puno Pinza chanchito 
paquete x 144 unidades, precio: 10 soles, stock: 800 paquetes.

Tienda Cuzco Pinza gatito 
paquete x 60 unidades, precio: 12 soles, stock: 500 paquetes.

Tienda Puno Pinza gatito 
paquete x 60 unidades, precio: 12 soles, stock: 500 paquetes.

Tienda Puno Cooler Freezer 
unidad, precio: 80 soles, stock 50 unidades.


In [28]:
# Importamos NLTK
import nltk
#Instalar solo una vez!!!
nltk.download('punkt')
nltk.download('wordnet') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## PRE PROCESADO

In [29]:
sent_tokens = nltk.sent_tokenize(raw_g) #converts to list of sentences 
word_tokens = nltk.word_tokenize(raw_g) #converts to list of words

len(sent_tokens) , len(word_tokens)

(9, 168)

In [30]:
# Obervamos las oraciones reconocidas
sent_tokens

['Tienda Cuzco Vaso tropical \npaquete x 18 unidades, precio: 16 soles, stock 300 paquetes.',
 'Tienda Humboldt Vaso tropical \npaquete x 12 unidades, precio: 12 soles, stock: 400 paquetes.',
 'Tienda Puno Silla robusta alta \npaquete x 6 unidades, precio:  114 soles, stock: 500 paquetes.',
 'Tienda Cuzco Banco robusto chato \npaquete x 12 unidades, precio: 104 soles, stock: 200 paquetes.',
 'Tienda Humboldt Pinza cucharita \npaquete x 144 unidades, precio: 9 soles, stock: 1000 paquetes.',
 'Tienda Puno Pinza chanchito \npaquete x 144 unidades, precio: 10 soles, stock: 800 paquetes.',
 'Tienda Cuzco Pinza gatito \npaquete x 60 unidades, precio: 12 soles, stock: 500 paquetes.',
 'Tienda Puno Pinza gatito \npaquete x 60 unidades, precio: 12 soles, stock: 500 paquetes.',
 'Tienda Puno Cooler Freezer \nunidad, precio: 80 soles, stock 50 unidades.']

In [31]:
## Elimino las puntuaciones del texto y conservo solo las palabras base
from nltk.stem import WordNetLemmatizer
import string

lemmer = WordNetLemmatizer()
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

#El lemma es la forma base de una palabra
def obtener_lemmas(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

def remover_caracteres(text):
    tokens = nltk.word_tokenize(text.translate(remove_punct_dict))
    return obtener_lemmas(tokens)


print(word_tokens[0:20])
print(remover_caracteres(raw_g)[0:20])

['Tienda', 'Cuzco', 'Vaso', 'tropical', 'paquete', 'x', '18', 'unidades', ',', 'precio', ':', '16', 'soles', ',', 'stock', '300', 'paquetes', '.', 'Tienda', 'Humboldt']
['Tienda', 'Cuzco', 'Vaso', 'tropical', 'paquete', 'x', '18', 'unidades', 'precio', '16', 'sol', 'stock', '300', 'paquetes', 'Tienda', 'Humboldt', 'Vaso', 'tropical', 'paquete', 'x']


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import warnings
warnings.filterwarnings('ignore')

GREETING_INPUTS = ("hola", "buenos dias","que tal",)
GREETING_RESPONSES = ["hi", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):
    """If user's input is a greeting, return a greeting response"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)


# Generating response
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=remover_caracteres, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Lo siento, no encuentro una respuesta, intenta con un producto en singular"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [0]:
#chooses a random message to send to the user. Here goes the NLP
def get_message():
  # user_response = "Hola... dime que producto deseas adquirir y en que tienda para mostrarte el stock"
  user_response = input()
  respuesta_bot = response(user_response)
  sent_tokens.remove(user_response)
  return respuesta_bot

# CORREMOS NGROK

In [36]:
import socket
print("http://" + socket.gethostbyname(socket.getfqdn(socket.gethostname())))

from time import sleep
sleep(1)

import threading
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80}).start()

sleep(5)

http://172.28.0.2


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 841, in run
    run_simple(host, port, self, **options)
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/serving.py", line 1009, in run_simple
    inner()
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/serving.py", line 962, in inner
    fd=fd,
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/serving.py", line 820, in make_server
    host, port, app, request_handler, passthrough_errors, ssl_context, fd=fd
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/serving.py", line 698, in __init__
    HTTPServer.__init__(self, server_address, handler)
  File "/usr/lib/python3.6/socketserver.py", line 456, in __init__
    self.server_bind()
  File "

In [15]:
print("La url de la politica de privacidad es: {}\nLa url de ngrok es: {}".format("https://drive.google.com/open?id=16JJXdpDTC6WPnex9fhlIhH7OKh5T3Shw", ngrok_ip))

La url de la politica de privacidad es: https://drive.google.com/open?id=16JJXdpDTC6WPnex9fhlIhH7OKh5T3Shw
La url de ngrok es: https://5f16d6eb.au.ngrok.io


In [16]:
import requests
r = requests.get(ngrok_ip)
print("Status code: {}, Encoding: {}, Apparent Encoding: {}, Text: {}".format(r.status_code, r.encoding, r.apparent_encoding, r.text))

127.0.0.1 - - [23/Sep/2019 22:46:50] "GET / HTTP/1.1" 200 -


Status code: 200, Encoding: utf-8, Apparent Encoding: ascii, Text: Invalid verification token
